In [59]:
#Importing read-sm-files.py
import ReadSMFIles

#reading j30.sm/j301_1.sm file
sm_file = ReadSMFIles.SMFileParser.parse_sm_file("j30.sm/j301_1.sm")

print(sm_file[3]['jobnr'])


0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    20
20    21
21    22
22    23
23    24
24    25
25    26
26    27
27    28
28    29
29    30
30    31
31    32
Name: jobnr, dtype: int64



Modelagem


In [46]:
#Dados
n = 0 #numero de trabalhos
m = 0 #número de tipos de recursos

